In [5]:
import pandas as pd
import os

In [ ]:
# Get all pathogens i.e. {pathogen}_{target}
PATHOGENS = sorted(os.listdir(os.path.join("..", "data")))

# Load all correlations
correlations = pd.read_csv(os.path.join("../output/05_correlations/05_correlations.tsv"), sep='\t')

In [118]:
def keep_minimum_exps(report):

    pchembl_value, percentile, percentage_activity, grouped_percentiles = {}, {}, {}, {}

    for task in report['task'].tolist():

        if "_percentile_" in task:
            lab = task.split("_percentile_")[0]
            if lab not in percentile:
                percentile[lab] = []
            percentile[lab].append(int(task.split("_percentile_")[1].split("_")[0]))

        elif "_pchembl_value_" in task:
            lab = task.split("_pchembl_value_")[0]
            if lab not in pchembl_value:
                pchembl_value[lab] = []
            pchembl_value[lab].append(int(task.split("_pchembl_value_")[1].split("_")[0]))

        elif "_percentage_activity_" in task:
            lab = task.split("_percentage_activity_")[0]
            if lab not in percentage_activity:
                percentage_activity[lab] = []
            percentage_activity[lab].append(int(task.split("_percentage_activity_")[1].split("_")[0]))

        elif "_grouped_percentiles_" in task:
            lab = task.split("_grouped_percentiles_")[0]
            if lab not in grouped_percentiles:
                grouped_percentiles[lab] = []
            grouped_percentiles[lab].append(int(task.split("_grouped_percentiles_")[1].split("_")[0]))

        else:
            print("Warning! Missing assignment for task:", task)
            break

    keep = []
    print(pchembl_value)

    for task in report['task'].tolist():
        
        if "_percentile_" in task:
            lab = task.split("_percentile_")[0]
            motif = lab + "_percentile_" + str(int(min(percentile[lab]))) + "_"
        
        elif "_pchembl_value_" in task:
            lab = task.split("_pchembl_value_")[0]
            motif = lab + "_pchembl_value_" + str(int(min(pchembl_value[lab]))) + "_"

        elif "_percentage_activity_" in task:
            lab = task.split("_percentage_activity_")[0]
            motif = lab + "_percentage_activity_" + str(int(min(percentage_activity[lab]))) + "_"

        elif "_grouped_percentiles_" in task:
            lab = task.split("_grouped_percentiles_")[0]
            motif = lab + "_grouped_percentiles_" + str(int(min(grouped_percentiles[lab]))) + "_"

        else:
            print("Warning! Missing assignment for task:", task)
            motif = "Not found"

        if motif in task:
            keep.append(True)
        else:
            keep.append(False)

    return keep

In [124]:
# Prioritized datasets
prioritized = []

# For each pathogen
for pathogen in PATHOGENS[:1]:

    # Load dataset report
    PATH_TO_CAMT = f"/home/acomajuncosa/Documents/chembl-antimicrobial-tasks/output/{pathogen}/018_selected_tasks_FINAL.csv"
    report = pd.read_csv(PATH_TO_CAMT)

    # Only those having MOD > 0.7
    report = report[report["auroc_avg_MOD"] > 0.7].reset_index(drop=True)

    # Only those having > 1000 samples
    report = report[report['num_samples_MOD'] > 1000].reset_index(drop=True)

    # Only those having > 100 positives
    report = report[report['num_pos_samples'] > 100].reset_index(drop=True)

    # Keeping min percentile/perc_activity/pchembl_value/grouped_percentiles per assay/target
    keep = keep_minimum_exps(report)
    report['keep'] = keep
    report = report[report['keep'] == True].reset_index(drop=True)

    # Only those having at least 1 correlation > 0.5
    correlations_pathogen = correlations[(correlations['Pathogen1'] == pathogen) & 
                                         (correlations['Pathogen2'] == pathogen) &
                                         (correlations['Model1'] == "RF") & 
                                         (correlations['Model2'] == "RF") & 
                                         (correlations['Same task'] == False)].reset_index(drop=True)
    # correlations_pathogen[correlations_pathogen['Spearman statistic'] > 0.5].reset_index(drop=True)
    # corr = set(correlations_pathogen['Task1'].tolist() + correlations_pathogen['Task2'].tolist())
    # corr = set(["_".join(i.split("_")[:-1]) for i in corr])
    # report = report[report['task'].isin(corr)].reset_index(drop=True)

    # # Sort by priority first, then by number of positive samples
    # report = report.sort_values(by=['priority', 'num_pos_samples'], ascending=[True, False]).reset_index(drop=True)

    # Store results
    # TOP = 3
    # prioritized.extend([[pathogen, i] for i in report['task'].tolist()[:TOP]])

    break

# prioritized = pd.DataFrame(prioritized, columns=['Pathogen', 'Task'])

{'2_target_CHEMBL614425_mic_um': [5, 6], '3_all': [7]}


In [126]:
correlations_pathogen

,Pathogen1,Task1,Model1,Pathogen2,Task2,Model2,Spearman statistic,Spearman pvalue,Pearson statistic,Pearson pvalue,Kendall statistic,Kendall pvalue,Same pathogen,Same task,Same model
0,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,0.715976,0.000000e+00,0.624602,0.000000e+00,0.532965,0.000000e+00,True,False,True
1,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,-0.253953,1.005974e-174,-0.196739,2.831306e-104,-0.169547,1.470345e-169,True,False,True
2,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,0.918818,0.000000e+00,0.910530,0.000000e+00,0.763619,0.000000e+00,True,False,True
3,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,abaumannii_organism,1_assay_CHEMBL4296188_MIC_pchembl_percentile_1...,RF,0.162216,4.677668e-71,0.133934,8.118881e-49,0.109974,5.363660e-72,True,False,True
4,abaumannii_organism,1_assay_CHEMBL4296188_Inhibition_percentage_ac...,RF,abaumannii_organism,1_assay_CHEMBL4296188_MIC_pchembl_percentile_1...,RF,0.130144,3.607180e-46,0.037962,3.400579e-05,0.101627,1.409649e-46,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,abaumannii_organism,4_all_percentage_activity_90_ORGANISM_1,RF,abaumannii_organism,4_all_percentage_activity_percentile_5_ORGANISM_1,RF,0.510625,0.000000e+00,0.339751,1.100185e-319,0.393347,0.000000e+00,True,False,True
296,abaumannii_organism,4_all_percentage_activity_90_ORGANISM_1,RF,abaumannii_organism,5_grouped_percentiles_1_ORGANISM_1,RF,0.467544,0.000000e+00,0.217285,2.603083e-127,0.357466,0.000000e+00,True,False,True
297,abaumannii_organism,4_all_percentage_activity_percentile_1_ORGANISM_1,RF,abaumannii_organism,4_all_percentage_activity_percentile_5_ORGANISM_1,RF,0.649607,0.000000e+00,0.560865,0.000000e+00,0.500641,0.000000e+00,True,False,True
298,abaumannii_organism,4_all_percentage_activity_percentile_1_ORGANISM_1,RF,abaumannii_organism,5_grouped_percentiles_1_ORGANISM_1,RF,0.636750,0.000000e+00,0.417454,0.000000e+00,0.484707,0.000000e+00,True,False,True


In [105]:
report['task'].tolist()

['1_assay_CHEMBL4296188_Inhibition_percentage_activity_percentile_1_ORGANISM',
 '1_assay_CHEMBL4296188_Inhibition_percentage_activity_percentile_5_ORGANISM',
 '1_assay_CHEMBL4296188_Inhibition_percentage_activity_percentile_10_ORGANISM',
 '2_target_CHEMBL614425_mic_um_pchembl_value_5_ORGANISM',
 '2_target_CHEMBL614425_mic_um_pchembl_value_6_ORGANISM',
 '2_target_CHEMBL614425_mic_um_pchembl_percentile_5_ORGANISM',
 '2_target_CHEMBL614425_inhibition_%_percentage_activity_percentile_1_ORGANISM',
 '3_all_pchembl_value_7_ORGANISM',
 '4_all_percentage_activity_percentile_1_ORGANISM',
 '4_all_percentage_activity_percentile_5_ORGANISM',
 '5_grouped_percentiles_1_ORGANISM']